In [1]:
import asyncio
# pip install transition
import transitions
# pip install nest-asyncio
import nest_asyncio
nest_asyncio.apply()

In [10]:
from transitions.extensions.asyncio import HierarchicalAsyncMachine
from transitions.extensions.asyncio import HierarchicalMachine

In [17]:
class BladderFlag():
    def __init__(self):

        self.machine = HierarchicalMachine(self, states=[], transitions = [])
        self.machine.add_states(['full', 'empty', 'nonempty'], ignore_invalid_triggers=False)
        self.machine.add_transition('to_full', ['initial', 'nonempty'], 'full')
        self.machine.add_transition('to_empty', ['initial', 'nonempty'], 'empty')
        self.machine.add_transition('to_nonempty', ['initial', 'full', 'empty'], 'nonempty')


    def add_sample(self, sample_arr):
        print(sample_arr)
        # if self.state == 'initial'
        n = int(sample_arr)
        if n==0: self.to_nonempty()
        elif n==1: self.to_empty()
        elif n==2: self.to_full()
        

In [18]:
bf = BladderFlag()

In [25]:
bf.state

'nonempty'

In [24]:
bf.add_sample(0)

0


In [3]:
import lib.comm_udp as com
import cfg.configuration as cfg


In [4]:
import time

In [5]:
from lib.logger import Logger


In [6]:
logger = Logger(cfg.app['test_mode'])
log = logger.get_log()
log.info('init')

In [7]:
class talker:
    def __init__(self, log) -> None:
        # self.model = AsyncModel()
        self.log = log
        self.transition = dict(trigger="start", source="Start", dest="Done", prepare="prepare_model",
                    before=["before_change"] * 5 + ["sync_before_change"],
                    after="after_change")  # execute before function in asynchronously 5 times
        self.states = ["Start", "Done"]
        self.machine = HierarchicalAsyncMachine(self, states=self.states, transitions=[self.transition], initial='Start')

    def prepare_model(self):
        print("I am synchronous.")
        self.start_time = time.time()
        self.comm = com.UdpComm("SIMULATION", cfg.app["simulation_udp_port"], self.log)

    async def before_change(self):
        print("I am asynchronous and will block now for 100 milliseconds.")
        await asyncio.sleep(0.1)
        print("I am done waiting.")

    def sync_before_change(self):
        print("I am synchronous and will block the event loop (what I probably shouldn't)")
        time.sleep(0.1)
        print("I am done waiting synchronously.")
        serial_line = self.comm.read() # e.g. data=['P1', '1.23']
        print(serial_line)
        self.sync_before_change()


    def after_change(self):
        print(f"I am synchronous again. Execution took {int((time.time() - self.start_time) * 1000)} ms.")

In [ ]:
ac = talker(log)


# asyncio.get_event_loop().run_until_complete(model.start())
asyncio.get_event_loop().run_until_complete(ac.start())
assert ac.is_Done()

asyncio.get_event_loop().create_datagram_endpoint

In [ ]:
asyncio.get_event_loop().stop()

In [3]:
import socket
import asyncio
import os, random

import nest_asyncio
nest_asyncio.apply()

HOST, PORT = 'localhost', 12000


def send_test_message(message: 'Message to send to UDP port 514') -> None:
    sock = socket.socket(socket.AF_INET,  # Internet
                         socket.SOCK_DGRAM)  # UDP
    sock.sendto(message.encode(), (HOST, PORT))


async def write_messages() -> "Continuously write messages to UDP port 514":
    dir_path = os.path.dirname(os.path.realpath(__file__))
    fp = open(os.path.join(dir_path, "tests/example.log"))
    print("writing")
    for line in fp.readlines():
        await asyncio.sleep(random.uniform(0.1, 3.0))
        send_test_message(line)


class SyslogProtocol(asyncio.DatagramProtocol):
    def __init__(self):
        super().__init__()

    def connection_made(self, transport) -> "Used by asyncio":
        self.transport = transport

    def datagram_received(self, data, addr) -> "Main entrypoint for processing message":
        # Here is where you would push message to whatever methods/classes you want.
        print(f"Received Syslog message: {data}")


if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    t = loop.create_datagram_endpoint(SyslogProtocol, local_addr=('0.0.0.0', PORT))
    loop.run_until_complete(t) # Server starts listening
    # loop.run_until_complete(write_messages()) # Start writing messages (or running tests)
    loop.run_forever()

OSError: [WinError 10048] Only one usage of each socket address (protocol/network address/port) is normally permitted

Received Syslog message: b'BT1:23.652142499382915'
Received Syslog message: b'BT2:23.124406720675523'
Received Syslog message: b'TT1:23.30674734559792'
Received Syslog message: b'TT2:23.357464247711906'
Received Syslog message: b'X:0.010001344851947215'
Received Syslog message: b'Y:0.0'
Received Syslog message: b'Z:0.0'
Received Syslog message: b'BP1:1051.3821022333764'
Received Syslog message: b'BP2:1055.0558829845324'
Received Syslog message: b'TP1:1003.201630837568'
Received Syslog message: b'TP2:1004.6277479376917'
Received Syslog message: b'HP:0.0'
Received Syslog message: b'PD:80.01071047988466'
Received Syslog message: b'PC:0.0'
Received Syslog message: b'AT:0.0'
Received Syslog message: b'AP:0.0'
Received Syslog message: b'BV:325.0775943317662'
Received Syslog message: b'HL:0'
Received Syslog message: b'EL:0'
Received Syslog message: b'BF:0'
Received Syslog message: b'BT1:23.671437540016342'
Received Syslog message: b'BT2:23.14841973946845'
Received Syslog message: b'TT1:23.283